### Подключим необходимые библиотеки

In [2]:
from sklearn.preprocessing import FunctionTransformer

from reskit.norms import binar_norm, wbysqdist
from reskit.norms import spectral_norm

from reskit.features import degrees,  pagerank

from reskit.core import Transformer, Pipeliner

from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import MinMaxScaler

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier 
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
import pandas as pd
import numpy as np

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

def orig(x):
    return x

### Функция считывания данных

In [3]:
def get_autism(path_to_read='Data/dti/', distances=True):
    def get_autism_distances(loc_name):
        with open(loc_name, 'r') as f:
            read_data = f.readlines()

        read_data = pd.DataFrame(
            np.array([np.array(item[:-1].split()).astype(int) for item in read_data]))

        return read_data

    def get_distance_matrix(coords):
        if type(coords) == pd.core.frame.DataFrame:
            coords = coords.values
        elif type(coords) != np.ndarray:
            print('Provide either pandas df or numpy array!')
            return -1

        shape = len(coords)
        dist_matrix = np.zeros((shape, shape))
        del shape
        for i in range(len(coords)):
            for j in range(i + 1, len(coords)):
                dist_matrix[i, j] = np.linalg.norm(coords[i, :] - coords[j, :])
                dist_matrix[j, i] = dist_matrix[i, j]
        return dist_matrix

    target_vector = []  # this will be a target vector (diagnosis)
    matrices = []  # this will be a list of connectomes
    all_files = sorted(os.listdir(path_to_read))
    matrix_files = [
        item for item in all_files if 'DTI_connectivity' in item and 'All' not in item]
    distance_files = [
        item for item in all_files if 'DTI_region_xyz_centers' in item and 'All' not in item]

    # for each file in a sorted (!) list of files:
    for filename in matrix_files:

        A_dataframe = pd.read_csv(
            path_to_read + filename, sep='   ', header=None, engine='python')
        A = A_dataframe.values  # we will use a list of numpy arrays, NOT pandas dataframes
        matrices.append(A)# append a matrix to our list
        if "ASD" in filename:
            target_vector.append(1)
        elif "TD" in filename:
            target_vector.append(0)
    asd_dict = {}
    asd_dict['X'] = np.array(matrices)
    asd_dict['y'] = np.array(target_vector)
    if distances:
        dist_matrix_list = []
        for item in distance_files:
            # print(item)
            cur_coord = get_autism_distances(path_to_read + item)
            cur_dist_mtx = get_distance_matrix(cur_coord)
            dist_matrix_list += [cur_dist_mtx]

        asd_dict['dist'] = np.array(dist_matrix_list)

    return asd_dict


### Функция понижения ранга матрицы

In [5]:
# Для первоночальной настройки ядра

from math import exp

def low(A, k):
    curs, vecs = np.linalg.eig(A)
    curs_abs = abs(curs)
    indeces_del = curs_abs.argsort()[:k]
    vecs_n = np.delete(vecs, indeces_del, axis=1)
    curs = np.delete(curs, indeces_del)
    #vecs_i = np.delete(np.linalg.inv(vecs), indeces_del, axis=0)
    A = vecs_n.dot(np.diag(curs)).astype('float')
    return A

def ker(data, i, j, k = 30):
    A = low(data['X'][i], k)
    B = low(data['X'][j], k)
    d = np.linalg.norm(A.dot(A.T) - B.dot(B.T), 'fro')
    #print "A: {}, B: {}, d = {}".format(A.shape, B.shape, d)
    return exp(-d * 10**-6)

In [6]:
#dist = np.zeros((94, 94), dtype = 'float')

#%%time
#for i in range(94):
#    for j in range(i+1):
#       dist[i][j] = ker(data, i, j, 255)
#        dist[j][i] = dist[i][j]

## Обучим SVM

In [4]:
def matrix_eig(data, k = 30):
    #new_data = {}
    #new_data['y'] = data['y']
    #new_data['dist'] = data['dist']
    #new_data['X'] = np.zeros(shape = (data['X'].shape[0], data['X'].shape[1], data['X'].shape[1] - k))
    for i in np.arange(data['X'].shape[0]):
        curs, vecs = np.linalg.eig(data['X'][i])
        curs_abs = abs(curs)
        indeces_del = curs_abs.argsort()[:k]
        vecs_n = np.delete(vecs, indeces_del, axis=1)
        curs = np.delete(curs, indeces_del)
        vecs_i = np.delete(np.linalg.inv(vecs), indeces_del, axis=0)
        data['X'][i] = vecs_n.dot(np.diag(curs)).dot(vecs_i).astype('float')
    return orig_vec(data)

def orig_vec(data):
    matrix = []
    for i in  data['X']:
        matrix.append(np.hstack(i))
    data['X_vec'] = matrix
    return data

def convert(A, mode, size = 264):
    if mode == 'mat2vec':
        A_vec = np.hstack(A)
        return A_vec
        
    if mode == 'vec2mat':
        A_mat = []
        i = 0
        while i != A.shape[0]:
            A_str = A[0+i:size+i]
            A_mat.append(A_str)
            i += size
        A_mat = np.vstack(A_mat)
        return A_mat

In [8]:
#Ядро

def my_kernel(A, B):
    A = convert(A, 'vec2mat')
    B = convert(B, 'vec2mat')
    d = np.linalg.norm(A.dot(A.T) - B.dot(B.T), 'fro')
    return exp(-d * 10**-6)

### Обычным способом

In [9]:
data = orig_vec(get_autism())
X = data['X_vec']
y = data['y']
X_low = matrix_eig(data, k=255)['X_vec']

In [11]:
print "example: X: {}({}), y: {}".format(X[0],X[0].shape, y[0])

example: X: [  0.  23.  26. ...,   0.  61.   0.]((69696,)), y: 1


In [13]:
print "example: X: {}({}), y: {}".format(X_low[0],X_low[0].shape, y[0])

example: X: [  1.06302133   0.49439527   1.61100051 ...,   4.10690851  15.40864896
  10.46470533]((69696,)), y: 1


In [14]:
clf = SVC(kernel=my_kernel)

In [ ]:
clf.fit(X_low, y)

### Используя класс Papiliner

In [12]:
grid_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=0)

eval_cv = StratifiedKFold(n_splits=10,
                          shuffle=True,
                          random_state=1)

data = [('UCLAsource', Transformer(get_autism))]

weighters = [('origW', Transformer(orig)),
             #('binar', Transformer(binar_norm))
            ]


normalizers = [('origN', Transformer(orig)),
               #('spectral', Transformer(spectral_norm))
              ]

featurizers = [#('origF', Transformer(orig_vec, collect=['X_vec'])),
               ('low_rank_255', Transformer(matrix_eig, collect=['X_vec']))]

selectors = [('var_threshold', VarianceThreshold())]

scalers = [('minmax', MinMaxScaler()),
           ('origS', FunctionTransformer(orig))]


classifiers = [('SVC', SVC())]

steps = [('Data', data),
         ('Weighters', weighters),
         ('Normalizers', normalizers),
         ('Featurizers', featurizers),
         ('Selectors', selectors),
         ('Scalers', scalers),
         ('Classifiers', classifiers)]

banned_combos = [
                 ('LR', 'origS'),
                 ('SVC', 'origS'),
                 ('SGD', 'origS'),
                 ('RF', 'minmax'),
                 ('XGB', 'minmax')
                 ]

param_grid = dict(
    SVC=dict(
        C=[0.001, 0.01] + [i*0.1 for i in range(1,6)],
        degree=[2, 3, 4],
        kernel=[my_kernel],
        max_iter=[50, 100, 150],
    )
)

pipe = Pipeliner(steps, eval_cv=eval_cv, grid_cv=grid_cv, param_grid=param_grid, banned_combos=banned_combos)
pipe.plan_table

,Data,Weighters,Normalizers,Featurizers,Selectors,Scalers,Classifiers
0,UCLAsource,origW,origN,low_rank_255,var_threshold,minmax,SVC


In [ ]:
result = pipe.get_results('Data/dti/', caching_steps=['Data', 'Weighters', 'Normalizers', 'Featurizers'], scoring=['roc_auc'])

Removed previous results file -- results.csv.
Line: 1/1
